# Repeated Games With Mistakes

## Nikolas Skoufis, 23/10/15
## Supervisor: Julian Garcia

## Prisoner's Dilemma

* Game between two prisoners who can either cooperate or defect
* Can encode outcomes in a payoff matrix
* Nash equilibrium is for both players to defect
* Iterated prisoner's dilemma is multiple rounds of the prisoner's dilemma
* Best strategy is to always defect, unless the number of rounds are variable
* If rounds are variabe TFT is the best strategy (cf. Axelrod's tournaments)

## Expected payoff

* Can compute the expected value of the payoff between two strategies

$$\sum_{i=0}^{\infty} = \delta^i \pi_i$$

* Closed forms for simple pairs of strategies
* Quickly becomes difficult for non-deterministic strategies
* Mistakes complicate all of this!
* Which strategies are fault tolerant?

## Tools and software

* Github for version control
* Travis for CI, Coveralls for code coverage

<img src="CoverallsAndTravis.png">

* Nose for testing, Hypothesis for property based testing
* Consider some simple code that encodes and decodes text to/from some character encoding

```python
from hypothesis import given
from hypothesis.strategies import text

@given(text())
def test_decode_inverts_encode(s):
    assert decode(encode(s)) == s
```

## Implementation

* Need a way to simulate and analyse different strategies

In [13]:
from repeatedmistakes.strategies import SuspiciousTitForTat, TitForTat
from repeatedmistakes.repeatedgame import RepeatedGame

my_game = RepeatedGame(SuspiciousTitForTat, TitForTat)
simulation_results = my_game.simulate(10)
print("STFT: " + str(simulation_results[SuspiciousTitForTat]))
print("TFT: " + str(simulation_results[TitForTat]))

STFT: ['D', 'C', 'D', 'C', 'D', 'C', 'D', 'C', 'D', 'C']
TFT: ['C', 'D', 'C', 'D', 'C', 'D', 'C', 'D', 'C', 'D']


* All strategies inherit from a base `Strategy` class
* Arbitrary strategies can be simulated, including non-deterministic ones because history is stored

## Calculation strategies

### Monte Carlo

* Monte Carlo methods, single processor and multiprocessor (using `multiprocessing` module)

In [16]:
from repeatedmistakes.simulations_multiprocessed import simulate_payoff
from repeatedmistakes.repeatedgame import PrisonersDilemmaPayoff

# Fixed number of trials
fixed_payoff = simulate_payoff(SuspiciousTitForTat, TitForTat, PrisonersDilemmaPayoff(),
                               continuation_probability=0.9, mistake_probability=0.01, trials=1000)

# With estimator stdev
estimator_payoff = simulate_payoff(SuspiciousTitForTat, TitForTat, PrisonersDilemmaPayoff(),
                                   continuation_probability=0.9, mistake_probability=0.01, estimator_stdev=0.2)

print("Payoff for fixed number of trials: " + str(fixed_payoff))
print("Payoff with estimator stdev: " + str(estimator_payoff))

Payoff for fixed number of trials: (1.6091999999999995, 1.4486999999999997)
Payoff with estimator stdev: (1.5616166666666662, 1.4038166666666665)


### Smart brute force

* Computational method using a queue and bounding of terms
* Amenable to multiprocessing, but large overhead

```python
# Set up a variable for the expected payoff
expected_payoff = 0

# Set up a queue to hold the partial histories
q = Queue()

# Initialize the queue with an empty history, with probability 1
q.put((1, '', ''))

while not q.empty():

    # Get an item from the front of the queue
    item = q.get()

    # Set up Strategy objects with the given histories
    player_one = Strategy(item.history1)
    player_two = Strategy(item.history2)

    # Compute the moves that the strategies produce with the given histories, passing the opponent's history as well
    move_one = player_one.next_move(player_two.history)
    move_two = player_two.next_move(player_one.history)

    # Compute the probability of no mistakes occurring
    probability = item.probability * no_mistake_probability * continuation_probability

    # If this maximum possible term size is larger than the threshold
    if probability * max_payoff > epsilon:
        # Multiply this by the payoff from the outcome of a no-mistake round to find the term
        term = probability * payoff(move_one, move_two)
        # Add the term to the expected payoff
        expected_payoff += term
        # Add the probability along with the histories (including the new moves) back onto the queue
        q.put(probability,
              item.history1 + move_one,
              item.history2 + move_two)

    else:
        # The probability was too small, so don't add it back to the queue

    # Repeat this for each of the two one mistake cases and the two mistake case
```

In [22]:
from repeatedmistakes.calculations_multiprocessed import calculate_payoff_with_mistakes

results = calculate_payoff_with_mistakes(SuspiciousTitForTat, TitForTat, PrisonersDilemmaPayoff(),
                                        continuation_probability=0.9, mistake_probability=0.1,
                                        epsilon=0.00001)

print("Smart brute force results: " + str(results))

Smart brute force results: (1.314917154047549, 1.17534565636084)


### Expected value only

* Similar to the last method, but only consider games with length = expected length
* Fast but not really that accurate

In [21]:
from repeatedmistakes.expected_only import expected_only

results = expected_only(SuspiciousTitForTat, TitForTat, PrisonersDilemmaPayoff(),
                        continuation_probability=0.9, mistake_probability=0.1,
                        epsilon=0.00001)

print("Expected value only results: " + str(results))

Expected value only results: (0.40908339981638664, 0.37768254183474503)


## Results

Forthcoming

## Conclusion

## Questions?

## References

* Ask me!